# Continuous training pipeline with KFP and Cloud AI Platform

**Learning Objectives:**
1. Learn how to use KF pre-build components (BiqQuery, CAIP training and predictions)
1. Learn how to use KF lightweight python components
1. Learn how to build a KF pipeline with these components
1. Learn how to compile, upload, and run a KF pipeline with the command line


In this lab, you will build, deploy, and run a KFP pipeline that orchestrates **BigQuery** and **Cloud AI Platform** services to train, tune, and deploy a **scikit-learn** model.


## Understanding the pipeline design


The workflow implemented by the pipeline is defined using a Python based Domain Specific Language (DSL). The pipeline's DSL is in the `covertype_training_pipeline.py` file that we will generate below.

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.


In [ ]:
from typing import NamedTuple

import kfp
from kfp import dsl

from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)

from kfp.v2.google.client import AIPlatformClient
from kfp.components import create_component_from_func_v2


from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

from jinja2 import Template

import time

In [ ]:
REGION = 'us-central1'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

ARTIFACT_STORE = f'gs://{PROJECT_ID}-vertex'

PIPELINE_ROOT = f'{ARTIFACT_STORE}/pipeline'
DATA_ROOT = f'{ARTIFACT_STORE}/data'
JOB_DIR_ROOT = f'{ARTIFACT_STORE}/jobs'
TRAINING_FILE_PATH = f'{DATA_ROOT}/training/dataset.csv'
VALIDATION_FILE_PATH = f'{DATA_ROOT}/validation/dataset.csv'
API_ENDPOINT = f'{REGION}-aiplatform.googleapis.com'

PIPELINE_NAME = 'covertype_kfp_pipeline'
PIPELINE_JSON = f'{PIPELINE_NAME}.json'

In [ ]:
SERVING_CONTAINER_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest'

IMAGE_NAME='trainer_image_covertype_vertex'
TAG='latest'
TRAINER_IMAGE=f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}'
TRAINER_IMAGE

In [ ]:
!gcloud builds submit --timeout 15m --tag $TRAINER_IMAGE trainer_image

In [ ]:
TIMESTAMP = time.strftime("%Y%m%d_%H%M%S")
JOB_NAME = f"covertype_training_{TIMESTAMP}"
JOB_DIR = f"{JOB_DIR_ROOT}/{JOB_NAME}"

STAGING_BUCKET = f'{PIPELINE_ROOT}/staging'

In [ ]:
def tune_hyperparameters(
        project: str,
        location: str,
        container_uri: str,
        training_file_path: str,
        validation_file_path: str,
        staging_bucket: str,
        job_dir: str,
        max_trial_count: int,
        parallel_trial_count: int
        
    ):
    from google.cloud import aiplatform
    from google.cloud.aiplatform import hyperparameter_tuning as hpt

    aiplatform.init(project=project, location=location, staging_bucket=staging_bucket)
    
    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
                "accelerator_type": "NVIDIA_TESLA_K80",
                "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": container_uri,
                "args": [
                    f"--job_dir={job_dir}",
                    f"--training_dataset_path={training_file_path}",
                    f"--validation_dataset_path={validation_file_path}",
                    "--hptune"                    
                ],
            },
        }
    ]
    
    custom_job = aiplatform.CustomJob(
        display_name='covertype_kfp_trial_job',
        worker_pool_specs=worker_pool_specs
    )
    
    hp_job = aiplatform.HyperparameterTuningJob(
        display_name='covertype_kfp_tuning_job',
        custom_job=custom_job,
        metric_spec={
            'accuracy': 'maximize',
        },
        parameter_spec={
            'alpha': hpt.DoubleParameterSpec(min=1.0e-4, max=1.0e-1, scale='linear'),
            'max_iter': hpt.DiscreteParameterSpec(values=[10, 20], scale='linear')
        },
        max_trial_count=max_trial_count,
        parallel_trial_count=parallel_trial_count,
    )

    hp_job.run()
    
    metrics = [trial.final_measurement.metrics[0].value for trial in hp_job.trials]
    best_trial = hp_job.trials[metrics.index(max(metrics))]
    best_alpha = best_trial.parameters[0].value
    best_max_iter = best_trial.parameters[1].value
    best_accuracy = best_trial.final_measurement.metrics[0].value
    
    return best_accuracy, best_alpha, best_max_iter

In [ ]:
tune_hyperparameters(
        project=PROJECT_ID,
        location=REGION,
        container_uri=TRAINER_IMAGE,
        training_file_path=TRAINING_FILE_PATH,
        validation_file_path=VALIDATION_FILE_PATH,
        staging_bucket=STAGING_BUCKET,
        job_dir=JOB_DIR,
        max_trial_count=5,
        parallel_trial_count=5,
)

In [ ]:
def train_and_deploy(
        project: str,
        location: str,
        container_uri: str,
        training_file_path: str,
        validation_file_path: str,
        staging_bucket: str,
        job_dir: str,
        alpha: float, 
        max_iter: int,
    ):
    from google.cloud import aiplatform
    
    
    SERVING_CONTAINER_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest'

    aiplatform.init(project=project, location=location,
                    staging_bucket=staging_bucket)
    job = aiplatform.CustomContainerTrainingJob(
        display_name='covertype_kfp_training',
        container_uri=container_uri,
        command=[
            "python", 
            "train.py",
            f"--job_dir={job_dir}",
            f"--training_dataset_path={training_file_path}",
            f"--validation_dataset_path={validation_file_path}",
            f"--alpha={alpha}",
            f"--max_iter={max_iter}",
            "--nohptune"
        ],
        staging_bucket=staging_bucket,
        model_serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    )
    model = job.run(replica_count=1, model_display_name='covertype_kfp_model')
    endpoint = model.deploy(
        traffic_split={"0": 100},
        machine_type='n1-standard-2',
    )

In [ ]:
tune_hpyerparameters_component = create_component_from_func_v2(
    tune_hyperparameters,
    base_image='python:3.8',
    output_component_file='covertype_kfp_tune_hyperparameters.yaml',
    packages_to_install=['google-cloud-aiplatform'],
)

train_and_deploy_component = create_component_from_func_v2(
    train_and_deploy,
    base_image='python:3.8',
    output_component_file='covertype_kfp_train_and_deploy.yaml',
    packages_to_install=['google-cloud-aiplatform'],

)

@dsl.pipeline(
    name="covertype-kfp-pipeline",
    description="The pipeline training and deploying the Covertype classifier",
    pipeline_root=PIPELINE_ROOT,
)
def covertype_train():
    
    tuning_op = tune_hpyerparameters_component(
        project=PROJECT_ID,
        location=REGION,
        container_uri=TRAINER_IMAGE,
        training_file_path=TRAINING_FILE_PATH,
        validation_file_path=VALIDATION_FILE_PATH,
        staging_bucket=STAGING_BUCKET,
        job_dir=JOB_DIR,
        max_trial_count=5,
        parallel_trial_count=5,
    )
    
    accuracy = tuning_op.outputs['best_accuracy']
    
    train_and_deploy_op = train_and_deploy(
        project=PROJECT_ID,
        location=REGION,
        container_uri=TRAINER_IMAGE,
        training_file_path=TRAINING_FILE_PATH,
        validation_file_path=VALIDATION_FILE_PATH,
        staging_bucket=STAGING_BUCKET,
        job_dir=JOB_DIR,
        alpha=tuning_op.outputs['best_alpha'], 
        max_iter=tuning_op.outputs['best_max_iter'],            
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func=covertype_train, 
    package_path=PIPELINE_JSON,
)

In [ ]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

In [ ]:
response = api_client.create_run_from_job_spec(
    job_spec_path=PIPELINE_JSON,
    # This argument is necessary if you did not specify PIPELINE_ROOT as part of the pipeline definition.
    # pipeline_root=PIPELINE_ROOT  
)

The custom components execute in a container image defined in `base_image/Dockerfile`.

In [ ]:
!cat base_image/Dockerfile

The training step in the pipeline employes the AI Platform Training component to schedule a  AI Platform Training job in a custom training container. The custom training image is defined in `trainer_image/Dockerfile`.

In [ ]:
!cat trainer_image/Dockerfile

## Building and deploying the pipeline

Before deploying to AI Platform Pipelines, the pipeline DSL has to be compiled into a pipeline runtime format, also refered to as a pipeline package.  The runtime format is based on [Argo Workflow](https://github.com/argoproj/argo), which is expressed in YAML. 


### Configure environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.
- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

### Build the trainer image

### Build the base image for custom components

In [ ]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE base_image

### Compile the pipeline

You can compile the DSL using an API from the **KFP SDK** or using the **KFP** compiler.

To compile the pipeline DSL using the **KFP** compiler.

#### Set the pipeline's compile time settings

The pipeline can run using a security context of the GKE default node pool's service account or the service account defined in the `user-gcp-sa` secret of the Kubernetes namespace hosting Kubeflow Pipelines. If you want to use the `user-gcp-sa` service account you change the value of `USE_KFP_SA` to `True`.

Note that the default AI Platform Pipelines configuration does not define the `user-gcp-sa` secret.

In [ ]:
USE_KFP_SA = False

COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/'
RUNTIME_VERSION = '1.15'
PYTHON_VERSION = '3.7'

%env USE_KFP_SA={USE_KFP_SA}
%env BASE_IMAGE={BASE_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

#### Use the CLI compiler to compile the pipeline

In [ ]:
!dsl-compile --py pipeline/covertype_training_pipeline.py --output covertype_training_pipeline.yaml

The result is the `covertype_training_pipeline.yaml` file. 

In [ ]:
!head covertype_training_pipeline.yaml

### Deploy the pipeline package

In [ ]:
PIPELINE_NAME='covertype_continuous_training'

!kfp --endpoint $ENDPOINT pipeline upload \
-p $PIPELINE_NAME \
covertype_training_pipeline.yaml

## Submitting pipeline runs

You can trigger pipeline runs using an API from the KFP SDK or using KFP CLI. To submit the run using KFP CLI, execute the following commands. Notice how the pipeline's parameters are passed to the pipeline run.

### List the pipelines in AI Platform Pipelines

In [ ]:
!kfp --endpoint $ENDPOINT pipeline list

### Submit a run

Find the ID of the `covertype_continuous_training` pipeline you uploaded in the previous step and update the value of `PIPELINE_ID` .


In [ ]:
PIPELINE_ID='defa3c60-637b-4332-88b9-d8647c2aec84'

In [ ]:
EXPERIMENT_NAME = 'Covertype_Classifier_Training'
RUN_ID = 'Run_001'
SOURCE_TABLE = 'covertype_dataset.covertype'
DATASET_ID = 'splits'
EVALUATION_METRIC = 'accuracy'
EVALUATION_METRIC_THRESHOLD = '0.69'
MODEL_ID = 'covertype_classifier'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

In [ ]:
!kfp --endpoint $ENDPOINT run submit \
-e $EXPERIMENT_NAME \
-r $RUN_ID \
-p $PIPELINE_ID \
project_id=$PROJECT_ID \
gcs_root=$GCS_STAGING_PATH \
region=$REGION \
source_table_name=$SOURCE_TABLE \
dataset_id=$DATASET_ID \
evaluation_metric_name=$EVALUATION_METRIC \
evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
model_id=$MODEL_ID \
version_id=$VERSION_ID \
replace_existing_version=$REPLACE_EXISTING_VERSION

where

- EXPERIMENT_NAME is set to the experiment used to run the pipeline. You can choose any name you want. If the experiment does not exist it will be created by the command
- RUN_ID is the name of the run. You can use an arbitrary name
- PIPELINE_ID is the id of your pipeline. Use the value retrieved by the   `kfp pipeline list` command
- GCS_STAGING_PATH is the URI to the GCS location used by the pipeline to store intermediate files. By default, it is set to the `staging` folder in your artifact store.
- REGION is a compute region for AI Platform Training and Prediction. 

You should be already familiar with these and other parameters passed to the command. If not go back and review the pipeline code.


### Monitoring the run

You can monitor the run using KFP UI. Follow the instructor who will walk you through the KFP UI and monitoring techniques.

To access the KFP UI in your environment use the following URI:

https://[ENDPOINT]


**NOTE that your pipeline run may fail due to the bug in a BigQuery component that does not handle certain race conditions. If you observe the pipeline failure, retry the run from the KFP UI**


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>